Name: Onur Alten
I hereby declare that I observed the honour code of the university when preparing the homework.

## Solution to 16.3-18

#### 16.3
p is the probability that trend continues,
q is the probability a new trend is set at a random y that breaks the previous trend.

\begin{equation*}
y_t=y_{t-1}=y_{t-2} \ with \ p \\
y_t\sim \upsilon(0,30) \ with \ 1-p \\
y_t=y_{t-1}\neq y_{t-2} \ with \ q \\
y_t=y_{t-2}\neq y_{t-1} \ with \ 1-q
\end{equation*}

Factorization:
\begin{equation*}
p(y_t|y_{t-1},y_{t-2})p(y_{t-1}|y_{t-2})p(y_{t-2})
\end{equation*}

#### 16.4
The alphabetic count resets with a probability p.

\begin{equation*}
x_t=x_{t-1}+1\ where \ x_{t-1}\leq 4 \ where  \ with \ p \\
x_t=x_{t-1} \ where \ x_{t-1}=5 with \ 1-p \\
x_t=1 \ with \ p
\end{equation*}

Factorization:
\begin{equation*}
p(x_t|x_{t-1})p(x_{t-1})
\end{equation*}

#### 16.5
Each trend last 3 or 4, then switch to the other character.

\begin{equation*}
p(x_t=x_{t-1}|x_{t-1}=x_{t-2}=x_{t-3})=p \\
p(x_t=|x_{t-1}-1| \ |x_{t-1}=x_{t-2}=x_{t-3})=1-p \\
x_t=|x_{t-1}-1| \ when \ x_{t-1}=x_{t-2}=x_{t-3}=x_{t-4} \\
x_t=x_{t-1} \ when \ x_{t-1}=x_{t-2}\neq x_{t-3} \\
x_t=x_{t-1} \ when \ x_{t-1}\neq x_{t-2}=x_{t-3}
\end{equation*}

Factorization:
\begin{equation*}
p(x_t|x_{t-1},x_{t-2},x_{t-3},x_{t-4})p(x_{t-1})p(x_{t-2})p(x_{t-3})p(x_{t-4})
\end{equation*}

#### 18.1
Directed Graphical Model:
<img src="Slide1.jpg\" style="width: 500px;\"> 
#### 18.2
Factor Graph:
<img src="Slide2.jpg\" style="width: 500px;\"> 
Undirected Graphical Model:
<img src="Slide3.jpg\" style="width: 500px;\"> 
#### 18.3
$ N-1+(N-1)*N+N-1+(N-1)*N+(N-1)*N+(N-1)*N*N+(N-1)*N+N+(N-1)*N*N = 2N^3+2N^2-2N-2 $

#### 18.4
a)Tr; p(A,B)=p(A)p(M)

b)Fa; X is A and M's descendant

c)Fa; X is T and L's descendant

d)Tr

e)Fa

In [7]:
import csv
import numpy as np
import pandas as pd

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T=[]
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

with open('transitions.csv','rb') as t:
    reader=csv.reader(t,delimiter=',')
    for row in reader:
        T.append(row)
T=np.array(T)
T=T.astype(np.float)

### 3.1
Generate random strings starting with '.' given the joint proability table.

In [6]:
def genString(first_char,N):
    x_prev=first_char
    string=x_prev
    
    for i in range(N):
        probs=T[letter2idx[x_prev]]                         #indexing the probability table row for x_{t-1}
        probs/=probs.sum()                                  #normalizing the indexed probability table row
        next_char=np.random.choice(alphabet,p=probs)        #randomly selects x_t with corresponding probabilities given x_{t-1}
        x_prev=next_char                                    
        string+=x_prev                                      #adds the most recently generated character to the string
    print string
                                                
genString('.',int(1+13*np.random.rand()))                #generating random length string starting with '.'

.nde.n


### 3.2

In [15]:
def findNearMin(array, val):
    diff=array-val                #difference between each element and given value
    positive=np.where(diff>=0)    #index of >0 elements in diff array
    return min(positive[0])       #return the min index that is larger than 0

def generateStringGiven(givenString):
    unknowns=np.empty(0)
    knowns=np.empty(0)
    diffs=np.empty(0)
    prevs=np.empty(0)
    nexts=np.empty(0)
    
    for idx,ch in enumerate(givenString):
        if ch=='_':
            unknowns=np.append(unknowns,idx)
    
    def pickChar(dist):
        dist=np.cumsum(dist, dtype=float)
        w=np.random.rand()
        pick=findNearMin(dist,w)
        return alphabet[pick]
    
    def getPrev(val):
        diff=knowns-val
        negative=np.where(diff<=0)[0]
        if negative.shape[0]==0:
            return None
        else:        
            return int(knowns[max(negative)])
        
    def getNext(val):
        diff=knowns-val
        positive=np.where(diff>=0)[0]
        if positive.shape[0]==0:
            return None
        else:        
            return int(knowns[min(positive)])                              
     
    def generate(previdx,nextidx):
        if previdx==None and nextidx!=None:
            n=1
            prevLetter='.'
            nextLetter=givenString[nextidx]
        elif nextidx==None and previdx!=None:
            n=len(givenString)-previdx-1
            nextLetter='.'
            prevLetter=givenString[previdx]
        else:
            n=nextidx-previdx-1
            prevLetter=givenString[previdx]
            nextLetter=givenString[nextidx]
        
        prev_dist=T[letter2idx[prevLetter]]
        next_dist=np.linalg.matrix_power(T,n).transpose()[letter2idx[nextLetter]]
        final_dist=prev_dist*next_dist
        final_dist/=np.sum(final_dist)
        return pickChar(final_dist)
    
    for j in unknowns:
        j=int(j)
        for idx,ch in enumerate(givenString):
            if ch=='_':
                unknowns=np.append(unknowns,idx)
            else:
                knowns=np.append(knowns,idx)

        prevGiven=getPrev(j)
        nextGiven=getNext(j)
        
        addChar=generate(prevGiven,nextGiven)
        s=list(givenString)
        s[j]=addChar
        givenString=''.join(s)
    print givenString
        
        
for k in range(len(test_strings)):
    print test_strings[k]
    generateStringGiven(test_strings[k])

th__br__n.f_x.
ththbr.an.fax.
_u_st__n_.to_be._nsw_r__
qu.streng.toube.answoror
i__at_._a_h_n_._e_r_i_g
is.ato.bathent.nearking
q___t.___z._____t.__.___.__.
qurut.yorz.we.tot.to.pre.as.


### 3.3

In [14]:
def generateStringGiven(givenString):
    unknowns=np.empty(0)
    knowns=np.empty(0)
    diffs=np.empty(0)
    prevs=np.empty(0)
    nexts=np.empty(0)
    
    for idx,ch in enumerate(givenString):
        if ch=='_':
            unknowns=np.append(unknowns,idx)
    
    def pickChar(dist):
        dist=np.cumsum(dist, dtype=float)
        w=np.random.rand()
        pick=findNearMin(dist,w)
        return alphabet[pick]
    
    def getPrev(val):
        diff=knowns-val
        negative=np.where(diff<=0)[0]
        if negative.shape[0]==0:
            return None
        else:        
            return int(knowns[max(negative)])
        
    def getNext(val):
        diff=knowns-val
        positive=np.where(diff>=0)[0]
        if positive.shape[0]==0:
            return None
        else:        
            return int(knowns[min(positive)])                              
     
    def generate(previdx,nextidx):
        if previdx==None and nextidx!=None:
            n=1
            prevLetter='.'
            nextLetter=givenString[nextidx]
        elif nextidx==None and previdx!=None:
            n=len(givenString)-previdx-1
            nextLetter='.'
            prevLetter=givenString[previdx]
        else:
            n=nextidx-previdx-1
            prevLetter=givenString[previdx]
            nextLetter=givenString[nextidx]
        
        prev_dist=T[letter2idx[prevLetter]]
        next_dist=np.linalg.matrix_power(T,n).transpose()[letter2idx[nextLetter]]
        final_dist=prev_dist*next_dist
        final_dist/=np.sum(final_dist)
        return alphabet[np.argmax(final_dist)]
    
    for j in unknowns:
        j=int(j)
        for idx,ch in enumerate(givenString):
            if ch=='_':
                unknowns=np.append(unknowns,idx)
            else:
                knowns=np.append(knowns,idx)

        prevGiven=getPrev(j)
        nextGiven=getNext(j)
        
        addChar=generate(prevGiven,nextGiven)
        s=list(givenString)
        s[j]=str(addChar)
        givenString=''.join(s)
    print givenString
        
        
for k in range(len(test_strings)):
    print test_strings[k]
    generateStringGiven(test_strings[k])
    print 
    

th__br__n.f_x.
the.br.an.fex.

_u_st__n_.to_be._nsw_r__
oursthend.to.be.answered

i__at_._a_h_n_._e_r_i_g
in.ath.wathend.he.r.ing

q___t.___z._____t.__.___.__.
qur.t.thiz.the.at.an.the.an.

